In [1]:
# If this task has dependencies, list them them here
# (e.g. upstream = ['some_task']), otherwise leave as None.
upstream = None

# This is a placeholder, leave it as None
product = None

In [2]:
# Parameters
query_params = {"rolling_window": 1, "tickers": ["TSLA"]}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/visualize_yfinance.ipynb"
}


In [3]:
# your code here...
import yfinance as yf
import pandas as pd
from src import utils
import altair as alt

alt.data_transformers.disable_max_rows()
alt.themes.enable('fivethirtyeight')
pd.set_option('display.float_format',  '{:,.2f}'.format)

In [4]:
tickers = query_params['tickers']
rolling_window = query_params['rolling_window']
# temp override
rolling_window = 1800 if rolling_window == 1 else rolling_window

start_date = utils.get_start_date(rolling_window)
start_date = utils.general_date_format(start_date)

In [5]:
ticker = yf.Ticker(tickers[0])

In [6]:
ticker_history = ticker.history(start=start_date)

In [7]:
ticker_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-04-18,59.94,60.17,59.58,60.05,15178500,0,0.00
2017-04-19,60.49,61.32,60.42,61.10,19490000,0,0.00
2017-04-20,61.30,61.83,60.05,60.50,30747000,0,0.00
2017-04-21,60.40,61.28,60.08,61.12,22549000,0,0.00
2017-04-24,61.84,62.11,61.20,61.61,25417500,0,0.00
...,...,...,...,...,...,...,...
2022-03-17,830.99,875.00,825.72,871.60,22194300,0,0.00
2022-03-18,874.49,907.85,867.39,905.39,33408500,0,0.00
2022-03-21,914.98,942.85,907.09,921.16,27327200,0,0.00


In [8]:
history = ticker_history.reset_index()
base = alt.Chart(history).configure(
    font='monospace'
)

In [9]:
base.mark_line().transform_calculate(
    validation = 'if(datum.Date > datetime("2021-03-01"), "True", "False")'
).encode(
    alt.X('Date:T'),
    alt.Y('Close'),
    alt.Color(
        'validation:O', 
        scale=alt.Scale(range=['green', 'orange']),
        legend=alt.Legend(
            values=['Traning', 'Validation'], 
            title='legend')
    )
).properties(
    width=900
)

alt.Chart(...)

In [10]:
base.mark_bar().transform_bin(
    'Stock Price (Binned)', 
    'Close',
    bin=alt.Bin(maxbins=25)
).encode(
        alt.Y('count()', title='Number of Days'),
    alt.X('Stock Price (Binned)', type='nominal'),
)

alt.Chart(...)

In [11]:
ticker.institutional_holders

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",62448572,2021-12-30,0.06,65994401918
1,Blackrock Inc.,52844995,2021-12-30,0.05,55845533816
2,Capital World Investors,36665256,2021-12-30,0.04,38747109235
3,State Street Corporation,32465720,2021-12-30,0.03,34309123581
4,"Geode Capital Management, LLC",13571554,2021-12-30,0.01,14342146836
5,"FMR, LLC",10988029,2021-12-30,0.01,11611929286
6,Baillie Gifford and Company,10798014,2021-12-30,0.01,11411125234
7,Jennison Associates LLC,10627050,2021-12-30,0.01,11230453899
8,Northern Trust Corporation,8558197,2021-12-30,0.01,9044131425
9,Price (T.Rowe) Associates Inc,8149225,2021-12-30,0.01,8611937995


In [12]:
ticker.mutualfund_holders

,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Total Stock Market Index Fund,22832068,2021-12-30,0.02,24128472821
1,Growth Fund Of America Inc,20852969,2021-11-29,0.02,23871644792
2,Vanguard 500 Index Fund,17226860,2021-12-30,0.02,18205001110
3,New Perspective Fund Inc,10649830,2021-12-30,0.01,11254527347
4,SPDR S&P 500 ETF Trust,9177630,2021-12-30,0.01,9698735831
5,"Invesco ETF Tr-Invesco QQQ Tr, Series 1 ETF",8669992,2021-12-30,0.01,9162274145
6,Fidelity 500 Index Fund,8038141,2021-12-30,0.01,8494546645
7,iShares Core S&P 500 ETF,6730436,2021-12-30,0.01,7112590156
8,Vanguard Growth Index Fund,6377167,2021-12-30,0.01,6739262542
9,Vanguard Institutional Index Fund-Institutiona...,6129162,2021-12-30,0.01,6477175818


In [13]:
ticker.splits

Date
2020-08-31   5.00
Name: Stock Splits, dtype: float64

In [14]:
ticker.info

{'zip': '78725',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 99290,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits. It provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; and purchase financing and leasing services. This segment is also involved in the provision of non-warranty after-sales vehicle services, sale of used vehicles, retail merchandise, and vehicle insurance, as well as sale of products to third party customers; services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians; and vehicle limited warrantie

In [15]:
ticker.cashflow

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Investments,"-132,000,000.00","-132,000,000.00","-132,000,000.00","-132,000,000.00"
Change To Liabilities,"5,371,000,000.00","2,423,000,000.00","1,447,000,000.00","2,203,000,000.00"
Total Cashflows From Investing Activities,"-7,868,000,000.00","-3,132,000,000.00","-1,436,000,000.00","-2,337,000,000.00"
Net Borrowings,"-5,732,000,000.00","-2,488,000,000.00","798,000,000.00","89,000,000.00"
Total Cash From Financing Activities,"-5,203,000,000.00","9,973,000,000.00","1,529,000,000.00","574,000,000.00"
Change To Operating Activities,"-3,014,000,000.00","-1,165,000,000.00","-1,000,000,000.00","-625,000,000.00"
Issuance Of Stock,"707,000,000.00","12,686,000,000.00","1,111,000,000.00","296,000,000.00"
Net Income,"5,519,000,000.00","721,000,000.00","-862,000,000.00","-976,000,000.00"
Change In Cash,"-1,757,000,000.00","13,118,000,000.00","2,506,000,000.00","312,000,000.00"
Effect Of Exchange Rate,"-183,000,000.00","334,000,000.00","8,000,000.00","-23,000,000.00"


In [16]:
ticker.balancesheet

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,"257,000,000.00","313,000,000.00","339,000,000.00","282,000,000.00"
Capital Surplus,"29,803,000,000.00","27,260,000,000.00","12,736,000,000.00","10,249,000,000.00"
Total Liab,"30,548,000,000.00","28,469,000,000.00","26,199,000,000.00","23,427,000,000.00"
Total Stockholder Equity,"30,189,000,000.00","22,225,000,000.00","6,618,000,000.00","4,923,000,000.00"
Minority Interest,"1,394,000,000.00","1,454,000,000.00","1,492,000,000.00","1,390,000,000.00"
Other Current Liab,"4,756,000,000.00","4,147,000,000.00","3,693,000,000.00","2,955,000,000.00"
Total Assets,"62,131,000,000.00","52,148,000,000.00","34,309,000,000.00","29,740,000,000.00"
Common Stock,"1,000,000.00","1,000,000.00","1,000,000.00",NaN
Other Current Assets,"345,000,000.00","238,000,000.00","246,000,000.00","193,000,000.00"
Retained Earnings,"331,000,000.00","-5,399,000,000.00","-6,083,000,000.00","-5,318,000,000.00"


In [17]:
ticker.analysis

,Max Age,End Date,Growth,Earnings Estimate Avg,Earnings Estimate Low,Earnings Estimate High,Earnings Estimate Year Ago Eps,Earnings Estimate Number Of Analysts,Earnings Estimate Growth,Revenue Estimate Avg,...,Revenue Estimate Growth,Eps Trend Current,Eps Trend 7Days Ago,Eps Trend 30Days Ago,Eps Trend 60Days Ago,Eps Trend 90Days Ago,Eps Revisions Up Last7Days,Eps Revisions Up Last30Days,Eps Revisions Down Last30Days,Eps Revisions Down Last90Days
Period,,,,,,,,,,,,,,,,,,,,,
0Q,1,2022-03-31,1.41,2.24,1.56,2.79,0.93,24.00,1.41,"17,571,900,000.00",...,0.69,2.24,2.24,2.24,2.08,1.77,0.00,0.00,1.00,NaN
+1Q,1,2022-06-30,0.70,2.47,1.46,3.13,1.45,23.00,0.70,"19,190,800,000.00",...,0.60,2.47,2.47,2.47,2.26,1.97,0.00,0.00,1.00,NaN
0Y,1,2022-12-31,0.55,10.50,6.89,14.15,6.78,32.00,0.55,"82,952,900,000.00",...,0.54,10.50,10.48,10.43,9.04,8.25,2.00,2.00,0.00,NaN
+1Y,1,2023-12-31,0.22,12.80,6.00,17.70,10.50,26.00,0.22,"104,445,000,000.00",...,0.26,12.80,12.73,12.73,11.70,10.53,2.00,2.00,0.00,NaN
+5Y,1,NaT,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5Y,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
ticker.calendar

,0,1
Earnings Date,2022-04-25 20:00:00,2022-04-29 20:00:00
Earnings Average,2.24,2.24
Earnings Low,1.56,1.56
Earnings High,2.79,2.79
Revenue Average,17571900000,17571900000
Revenue Low,14331000000,14331000000
Revenue High,19754800000,19754800000


In [19]:
ticker.earnings

,Revenue,Earnings
Year,,
2018,21461000000,-976000000
2019,24578000000,-862000000
2020,31536000000,721000000
2021,53823000000,5519000000


In [20]:
ticker.financials

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,"2,593,000,000.00","1,491,000,000.00","1,343,000,000.00","1,460,000,000.00"
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,"6,343,000,000.00","1,154,000,000.00","-665,000,000.00","-1,005,000,000.00"
Minority Interest,"1,394,000,000.00","1,454,000,000.00","1,492,000,000.00","1,390,000,000.00"
Net Income,"5,519,000,000.00","721,000,000.00","-862,000,000.00","-976,000,000.00"
Selling General Administrative,"4,517,000,000.00","3,188,000,000.00","2,646,000,000.00","2,835,000,000.00"
Gross Profit,"13,606,000,000.00","6,630,000,000.00","4,069,000,000.00","4,042,000,000.00"
Ebit,"6,523,000,000.00","1,951,000,000.00","80,000,000.00","-253,000,000.00"
Operating Income,"6,523,000,000.00","1,951,000,000.00","80,000,000.00","-253,000,000.00"
Other Operating Expenses,"-27,000,000.00","-27,000,000.00","-27,000,000.00","-27,000,000.00"


In [21]:
ticker.news

[{'uuid': 'f7354f6f-92c2-3d75-8402-1a6d13a5583f',
  'title': "Rep. Stewart talks Elon Musk's business ties in China, inflation, and energy outlook",
  'publisher': 'Yahoo Finance Video',
  'link': 'https://finance.yahoo.com/video/rep-stewart-talks-elon-musks-160450017.html',
  'providerPublishTime': 1648051490,
  'type': 'VIDEO'},
 {'uuid': '776f4f6c-26fd-3dca-b84d-880456c3e3f0',
  'title': 'Toyota (TM), ChargePoint Team Up for Multi-Pronged EV Charging',
  'publisher': 'Zacks',
  'link': 'https://finance.yahoo.com/news/toyota-tm-chargepoint-team-multi-142802649.html',
  'providerPublishTime': 1648045682,
  'type': 'STORY'},
 {'uuid': '3396bf99-773a-3dd8-8314-0b760a40db37',
  'title': 'Is it a Wise Move to Still Buy Tesla, Inc. (TSLA) Shares?',
  'publisher': 'Insider Monkey',
  'link': 'https://finance.yahoo.com/news/wise-move-still-buy-tesla-140510618.html',
  'providerPublishTime': 1648044310,
  'type': 'STORY'},
 {'uuid': 'eb488cf7-07b6-38b1-8a83-582c6dc9cf57',
  'title': 'The Zack

In [22]:
ticker.recommendations

,Firm,To Grade,From Grade,Action
Date,,,,
2012-02-16 07:42:00,JP Morgan,Overweight,,main
2012-02-16 13:53:00,Wunderlich,Hold,,down
2012-02-17 06:17:00,Oxen Group,Buy,,init
2012-03-26 07:31:00,Wunderlich,Buy,,up
2012-05-22 05:57:00,Maxim Group,Buy,,init
...,...,...,...,...
2022-01-27 11:53:32,Wells Fargo,Equal-Weight,,main
2022-01-27 14:14:58,Citigroup,Sell,,main
2022-01-31 10:10:29,Credit Suisse,Outperform,Neutral,up
